# Preparation of Dataset for Random Forest

@author: Caroline Gasten

For the RF model data from different sources needs to be merged. In the following each of the products is subsequently added to a common dataframe:
1. drought-conflict mapping (as also used on smalle spatial scale for logistic regression model)
2. monthly DataCube variables which have already been prepared for the time period of interest
3. other WPS Global Early Warning Tool variables which needed to be manually preprocessed
4. conflict input variables which have been prepared as part of the conflict data preparation

## Settings

In [ ]:
#import packages
import h5py
import os
import tables
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime

In [ ]:
#required paths
path_pv= r'C:\Users\gasten\OneDrive - Stichting Deltares\Documents\02_Data\Predictor Variables'#path to directory with all predictor variables
path_dc_mapping = r'C:\Users\gasten\OneDrive - Stichting Deltares\Documents\02_Data\06_Drought_Conflict_Mapping'#path to mapped drought-conflict data
path_conflict_input = r'C:\Users\gasten\OneDrive - Stichting Deltares\Documents\02_Data\04_Conflict_Data\RF_input'#path to conflict input variables

## Functions

In [ ]:
def year_from_unix(unix_date):
    """
    The present function obtains a date in unix format and retrieves the year from the date
    """
    unix_date_cropped = unix_date/1000000000
    date = datetime.datetime.fromtimestamp(unix_date_cropped)
    year = date.year
    return year

#vectorize the function to deal with arrays of unix dates
year_from_unix_vec = np.vectorize(year_from_unix)   

## Conflict & Drought Data

In [ ]:
#conflict & drought data
i=0

#loop through the 6 chosen drought indicators
for DI, dt in zip(['SPEI-1', 'SPI-1', 'SPI-3', 'SPEI-6', 'SPI-6', 'SPI-12'], [0, 7,2, 0,4,2]):
    
    #read data on spatial Median to dataframe
    df = pd.read_csv(os.path.join(path_dc_mapping,  'adm_%s_lag%i_conflict.csv'%(DI, dt)), parse_dates=['time']).drop('Unnamed: 0', axis=1).loc[:, ['time', 'county', 'GID_1', 'conflict_boolean', 'Median']].set_index(['time', 'GID_1'])
    
    #join the different DIs to one dataframe
    if i == 0:
        df_conflict_drought = df.copy()
    else:
        df_conflict_drought = df_conflict_drought.join(df['Median'], on=['time', 'GID_1'], how='inner', rsuffix='_%s_lag%i'%(DI, dt))
    i+=1

#coherent names for all columns
df_conflict_drought.rename({'Median': 'Median_SPEI-1_lag0'}, axis=1, inplace=True)

#reset index
df_conflict_drought.reset_index(inplace=True)


## Read Data from DataCube

In [ ]:
#initialize dataframe for random forest model
df_data_rf = pd.DataFrame()

In [ ]:
#open datastructure for monthly data
f_adm1_m = tables.open_file(os.path.join(path_pv, 'raw\data_working_phase_3_datacubes_Version_Archive_datacube_objects_v1.3.2_datacube_from_hdf5_month-admin1_merged.h5'), mode='r') #only monthly data is used from datacube as yearly data was not available for the correct time period, see separate script for reproduction of yearly variables from source data

In [ ]:
#retrieve data on administrative units
ADM_array = f_adm1_m.root.table.block3_values
ADM1_column = ADM_array[0][:,0]

#condition that adm1 units need to be in drought-conflict mapping dataframe
cond = np.isin(ADM1_column, df_conflict_drought.GID_1.unique())

df_data_rf['GID_1'] = ADM_array[0][cond,0]
df_data_rf['GID_0'] = ADM_array[0][cond,1]

In [ ]:
#loop through all variables in DataCube and add to dataframe
for i in range(3):
    block_name = 'block' + str(i)
    col_names = getattr(f_adm1_m.root.table, block_name + '_items')
    val_array = getattr(f_adm1_m.root.table, block_name + '_values')
    for j in range(len(col_names)):
        column = val_array[:,j]
        df_data_rf[col_names[j].decode('utf-8')] = column[cond]
df_data_rf = df_data_rf.set_index(['GID_1', 'year', 'month'])

## Join DataCube with Own Data

In [ ]:
#make indices for drought-conflict data the same
df_conflict_drought['year'] = df_conflict_drought.time.dt.year
df_conflict_drought['month'] = df_conflict_drought.time.dt.month
df_conflict_drought = df_conflict_drought.set_index(['GID_1', 'year', 'month'])

In [ ]:
#join drought-conflict data with monthly DataCube data
df_mapping = df_conflict_drought.join(df_data_rf)

## Add yearly variables

In [ ]:
#open data on prepared yearly predictor variables
df_yearly_vars = pd.read_csv(os.path.join(path_pv, "yearly_pred_vars.csv"))

In [ ]:
#introduce 'GID_0' column
df_yearly_vars['GID_0'] = df_yearly_vars.Country.map({'Ethiopia':'ETH', 'Kenya': 'KEN', 'South Sudan': 'SSD', 'Sudan': 'SDN', 'Somalia':'SOM', 'Uganda':'UGA'})

In [ ]:
#set index to GID_0 and year
df_yearly_vars = df_yearly_vars.set_index(["GID_0", "Year"]).drop(columns="Unnamed: 0")

In [ ]:
#join yearly variables with remaining variables -> constant value for all admin1 units within one country and all months of one year (compare to Kuzma et al. (2020))
df_mapping = df_mapping.join(df_yearly_vars, on=["GID_0", "year"])

## Add Conflict Input Variables

In [ ]:
#open dataframe with spillover conflict variables
df_spat_lag = pd.read_csv(os.path.join(path_conflict_input, 'spilloverconflict_t-1.csv'), parse_dates=['Unnamed: 0']).rename({'Unnamed: 0': 'time', 'com': 'spat_com_t-1', 'oc': 'spat_oc_t-1'},axis=1).set_index(['time', 'county'])

In [ ]:
#open dataframe with variable "time since last communal conflict"
df_dt_conflict = pd.read_csv(os.path.join(path_conflict_input, 'timesinceconflict.csv'), parse_dates=['Unnamed: 0']).rename({'Unnamed: 0': 'time', '0': 'dt_conflict'}, axis=1)

In [ ]:
#combine conflict input variables
df_conflict_vars = df_dt_conflict.join(df_spat_lag, on=['time', 'county'], how='inner')

In [ ]:
#include columns on year, month and administrative unit to match the format of dataframe with all other variables
df_conflict_vars['year'] = df_conflict_vars.time.dt.year
df_conflict_vars['month'] = df_conflict_vars.time.dt.month
df_conflict_vars = df_conflict_vars.set_index(['GID_1', 'year', 'month'])

In [ ]:
#join conflict input variables with other variables in dataframe
df_mapping = df_mapping.join(df_conflict_vars, on=['GID_1', 'year', 'month'], lsuffix = 'DC', how='inner')

## Retrieving needed variables for RF Model and storing them in csv file

In [ ]:
#variables which are included vs. excluded
vars_ds = ['conflict_boolean', 'Median_SPEI-1_lag0', 'Median_SPI-1_lag7', 'Median_SPI-3_lag2', 'Median_SPEI-6_lag0', 'Median_SPI-6_lag4', 'Median_SPI-12_lag2', 'loccount_y', 'locdensity_y', 'rurratio_s', 'spam_V_agg_r_sum_s', 'rfr_s', 'sev_s', 'iav_s', 'spat_com_t-1', 'spat_oc_t-1', 'dt_conflict', 'male_pct_65+', 'sex_ratio_25-64', 'sex_ratio_65+', 'sanitationaccess']
vars_missing = ['NY.GDP.PCAP.PP.KD', 'NY.AGR.TOTL.ZS']

In [ ]:
#retrieve variables to be included from whole dataframe
df_ml = df_mapping.loc[:,vars_ds]

In [ ]:
#store dataframe with conflict output variable and all input variables for RF model to csv file
df_ml.to_csv(os.path.join(path_pv, 'RF_dataframe_input.csv'))